## First version of ML model

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

from load_data import load_dance_data
from load_data import one_hot_encoder
from sklearn.utils import shuffle

import numpy as np
import os

In [2]:
# dataset = 'C:\\Users\\shrey\\Desktop\\NUS- Everything\\Semester 7\\CG4002\\CG4002_B18\\sw1_ml\\data\\processed\\'

dataset = os.getcwd() + '\\Dance_data\\'

sampling_rate = 5
window_size = 2.56

X, y = load_dance_data(dataset, sampling_rate, window_size)

Files read from  Hair.txt
Files read from  Rocket.txt
Files read from  Zig_Zag.txt


In [3]:
X, y = shuffle(X, y)
X_train, X_test = X[:int(len(X)*0.8), :], X[int(len(X)*0.8):, :]
y_train, y_test = y[:int(len(X)*0.8), :], y[int(len(X)*0.8):, :]

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
y_train = y_train.reshape(y_train.shape[0]).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
y_test = y_test.reshape(y_test.shape[0]).long()

input_size = X.shape[1]
output_size = one_hot_encoder(y).shape[1]

In [4]:
hidden_states = [256, 128]

model = nn.Sequential(nn.Linear(input_size, hidden_states[0]),
            nn.ReLU(),
            nn.Linear(hidden_states[0], hidden_states[1]),
            nn.ReLU(),
            nn.Linear(hidden_states[1], output_size),
            nn.LogSoftmax(dim=1))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [5]:
# Prepare data for training and testing
import torch.utils.data

class Prepare_data(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        x = self.inputs[index]
        y = self.labels[index]
        return x, y


In [6]:
trainloader = torch.utils.data.DataLoader(Prepare_data(X_train, y_train), batch_size=32, shuffle=True)

testloader = torch.utils.data.DataLoader(Prepare_data(X_test, y_test), batch_size=32)

In [7]:
epochs = 15

# Define the loss
criterion = nn.NLLLoss()
# Optimizers require the parameters to optimize and a learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.003)

for e in range(epochs):
    running_loss = 0
    for inputs, labels in trainloader:
    
        # Training pass
        optimizer.zero_grad()
        
        output = model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

Training loss: 184.7803331392152
Training loss: 1.0217469811439515
Training loss: 0.9954640456608363
Training loss: 0.9989692143031529
Training loss: 0.7252138529505049
Training loss: 0.5522320704800742
Training loss: 0.5006617035184587
Training loss: 2.4122658865792412
Training loss: 1.0826406734330314
Training loss: 2.0484613180160522
Training loss: 1.06397511107581
Training loss: 1.0354888473238264
Training loss: 1.0238720434052604
Training loss: 1.2717085804258075
Training loss: 1.0202201758112226


In [8]:
y_pred = model(X_test)

y_pred = y_pred.cpu()
result = y_pred.data.numpy()
array_res = np.reshape(result, (1, -1))

In [7]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import EarlyStopping

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def score_function(engine):
    val_loss = engine.state.metrics['crossentropy']
    accuracy = engine.state.metrics['accuracy']
    return accuracy

trainer = create_supervised_trainer(model, optimizer, loss)
evaluator = create_supervised_evaluator(model,
                                        metrics={
                                            'accuracy': Accuracy(),
                                            'crossentropy': Loss(loss)
                                            })

handler = EarlyStopping(patience=5, score_function=score_function, trainer=trainer)
# Note: the handler is attached to an *Evaluator* (runs one epoch on validation dataset).
evaluator.add_event_handler(Events.COMPLETED, handler)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(trainloader)
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['crossentropy']))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(testloader)
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['crossentropy']))

trainer.run(trainloader, max_epochs=35)

Training Results - Epoch: 1  Avg accuracy: 0.94 Avg loss: 0.18
Validation Results - Epoch: 1  Avg accuracy: 0.95 Avg loss: 0.18
Training Results - Epoch: 2  Avg accuracy: 1.00 Avg loss: 0.01
Validation Results - Epoch: 2  Avg accuracy: 1.00 Avg loss: 0.01
Training Results - Epoch: 3  Avg accuracy: 1.00 Avg loss: 0.00
Validation Results - Epoch: 3  Avg accuracy: 1.00 Avg loss: 0.00
Training Results - Epoch: 4  Avg accuracy: 1.00 Avg loss: 0.00
Validation Results - Epoch: 4  Avg accuracy: 1.00 Avg loss: 0.00


State:
	iteration: 140
	epoch: 4
	epoch_length: 35
	max_epochs: 35
	output: 0.00020066721481271088
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [14]:
torch.save(model.state_dict(), "NN_v1_3moves.pt")

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test inputs: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test inputs: 100 %


In [11]:
classes = ['Hair', 'Rocket', 'Zig-Zag']
dataiter = iter(testloader)
inputs, labels = dataiter.next()

print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(len(labels))))

outputs = model(inputs)

_, predicted = torch.max(outputs, 1)

print('Predicted:   ', ' '.join('%5s' % classes[predicted[j]] for j in range(len(labels))))


GroundTruth:   Hair  Hair  Hair Rocket Zig-Zag Rocket Zig-Zag  Hair  Hair  Hair Zig-Zag  Hair  Hair  Hair Rocket  Hair  Hair Zig-Zag Rocket  Hair Zig-Zag  Hair Zig-Zag  Hair Zig-Zag  Hair  Hair  Hair Rocket Rocket Rocket  Hair
Predicted:     Hair  Hair  Hair Rocket Zig-Zag Rocket Zig-Zag  Hair  Hair  Hair Zig-Zag  Hair  Hair  Hair Rocket  Hair  Hair Zig-Zag Rocket  Hair Zig-Zag  Hair Zig-Zag  Hair Zig-Zag  Hair  Hair  Hair Rocket Rocket Rocket  Hair


In [16]:
first_model = model
first_model.load_state_dict(torch.load("NN_v1_3moves.pt"))
first_model.eval()

Sequential(
  (0): Linear(in_features=40, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=3, bias=True)
  (5): LogSoftmax()
)

## Next Steps

* Employ additional Machine Learning Models and test out the accuracy obtained
* Update the feature extraction process to include better measures as well as the Gyroscope data
* Apply preprocessing on the dataset to remove noise
* Run the models on the online dataset